# Projektarbeit Data Analytics

Sebastian Jana,
Sophie Jana

## Inhaltsverzeichnis
<a id ="inhaltsverzeichnis"></a>

[1. Aufgabe](#aufgabe1)

2. Aufgabe

3. Aufgabe

4. Aufgabe
5. Aufgabe

[6. Aufgabe](#aufgabe6)

[7. Quellenverzeichnis](#quellenverzeichnis)




### Aufgabe 1 (Datenvorbereitung)
<a id = "aufgabe1"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a) Einlesen der CSV Dateien, welche die Stromerzeugungsdaten und die Börsenstrompreise enthalten, als Dataframe df_hourly.


In [3]:
import pandas as pd
import numpy as np

In [ ]:
import os
import glob

path = './Daten/Strompreisdaten'
csv_files = glob.glob(os.path.join(path, "*csv"))

# df erzeugen
df_list = []
for i in range(len(csv_files)):
    try:
        df_temp = pd.read_csv(csv_files[i], sep = ",")
        # df mit neuer file appenden
        df_temp = df_temp.rename(columns = {'Datum (MESZ)' : 'Datum'})
        df_temp = df_temp.rename(columns = {'Datum (MEZ)' : 'Datum'})
        df_list.append(df_temp)
    except Exception as e: 
        print("Fehler beim Einlesen des Files: ", csv_files[i])
    
df_hourly = pd.concat(df_list)
print(df_hourly)

df_neu = df_hourly[df_hourly["Datum (MEZ)"].notnull()]
# df_neu

### Aufgabe 6 (Analyse von Stromtarif-Angeboten für Endkunden)
<a id = "aufgabe6"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a)  Einlesen der JSON-Preisvergleichdaten in einem DataFrame df_cust


In [ ]:
path = './Daten/Endkundenpreise/'

def extract_date_from_filepath(file):
    file = file.replace(path, "")
    # Concatenate the date out of fixed year 2024 and month/day from folder name
    return "2024-" + file[0:5]

def extract_data_from_json(file):
    assert file.endswith(".json"), "Keine json-Datei uebergeben"
    try:
        # Read and load the json file
        df_temp = pd.read_json(json_files[i])
        # Transpose the table: convert the rows to columns
        df_temp = df_temp.T
        # Index = Number of entry in the json file
        df_temp['Angebotsnummer'] = df_temp.index
        # Add date from filename as column
        df_temp['Datum'] = extract_date_from_filepath(file)
        
        return df_temp
    except:
        print("Datei konnte nicht gelesen werden.")


path = './Daten/Endkundenpreise/'
df_list = []
# https://www.tutorialspoint.com/python/os_listdir.htm
for folder in os.listdir(path):
    combined_path = os.path.join(path, folder, "*json")
    json_files = glob.glob(combined_path)

    for i in range(len(json_files)):
        df_temp = extract_data_from_json(json_files[i])
        df_list.append(df_temp)
        
df_cust = pd.concat(df_list)
df_cust


### Quellenverzeichnis
<a id = "quellenverzeichnis"></a>

[1] 

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)